# Homework 10

In [89]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ISLP.models import ModelSpec as MS

In [90]:
Hitters = pd.read_csv('C:\\Users\\lorentsen\\Documents\\Python\\Math-180-Applied-Statistics-and-Machine-Learning\\ISLRv2_data\\ALL CSV FILES - 2nd Edition\\Hitters.csv')
Hitters.dropna(inplace=True)
y = Hitters['Salary']
x = Hitters.drop('Salary', axis=1)

## Problem 1
### part a)

In [91]:
x = x.astype({'AtBat': 'float64', 'Hits': 'float64', 'HmRun': 'float64', 'Runs': 'float64', 'RBI': 'float64', 'Walks': 'float64', 'Years': 'float64', 'CAtBat': 'float64', 'CHits': 'float64', 'CHmRun': 'float64', 'CRuns': 'float64', 'CRBI': 'float64', 'CWalks': 'float64', 'PutOuts': 'float64', 'Assists': 'float64', 'Errors': 'float64'})
for i in range(0, x.shape[1]):
    if x.iloc[:, i].dtype == 'object':
        continue
    x.iloc[:,i] = (x.iloc[:, i] - x.iloc[:, i].mean())/x.iloc[:, i].std()

y = (y - y.mean())/y.std()
x['NewLeague'] = x['NewLeague'].map({'A': 1, 'N': 0})
x['League'] = x['League'].map({'A': 1, 'N': 0})
x['Division'] = x['Division'].map({'E': 1, 'W': 0})
# print(x.head())


### part b)

In [92]:
import statsmodels.api as sm

In [93]:
# newmodel = MS(x, intercept = False)
# X = newmodel.fit_transform(x)
# Y = y
model = sm.OLS(y, x).fit()
model.params

AtBat       -0.658632
Hits         0.755723
HmRun        0.080603
Runs        -0.121588
RBI         -0.047869
Walks        0.291328
Years       -0.021399
CAtBat      -0.892974
CHits        0.138303
CHmRun      -0.033312
CRuns        1.132763
CRBI         0.581068
CWalks      -0.475808
League      -0.168323
Division     0.206751
PutOuts      0.174039
Assists      0.122905
Errors      -0.052294
NewLeague    0.020377
dtype: float64

### part c)

We have
\begin{equation}
f(X) = \beta_0 + \sum^K_{k=1} \beta_k h_k(X)
\end{equation}
where $h_k$ is our activation function, which is just connected to one predictor for each $k$. We can rewrite this as
\begin{equation}
f(X) = \beta_0 +\sum^p_{k=1} \beta_k X_k
\end{equation}
Which is just linear regression.

### part d)


In [94]:
newmodel = MS(x, intercept = False)
X = newmodel.fit_transform(x)
Y = y

# X['NewLeague'] = X['NewLeague'].map({'A': 1, 'N': 0})
# X['League'] = X['League'].map({'A': 1, 'N': 0})
# X['Division'] = X['Division'].map({'E': 1, 'W': 0})

### part e)

In [95]:
import keras
from keras import layers
from keras import ops

In [96]:
X.shape

(263, 19)

In [97]:
modelnn = keras.Sequential(
    [
        # Input(shape),
        layers.Dense(1, activation = None, input_shape = (X.shape[1],)),
    ]
)

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### part f)

In [98]:
modelnn.compile(loss='mean_squared_error', optimizer='adam')
modelnn.fit(X, Y, epochs=200, verbose=0)

### part g)

In [99]:
showparams = pd.DataFrame()
# print(model.params)
showparams['glm'] = model.params
showparams['nn'] = modelnn.get_weights()[0]
# showparams.index = model.params.index
print(showparams)

                glm        nn
AtBat     -0.658632  0.022274
Hits       0.755723 -0.091555
HmRun      0.080603 -0.314586
Runs      -0.121588  0.023879
RBI       -0.047869  0.586889
Walks      0.291328  0.002488
Years     -0.021399 -0.290138
CAtBat    -0.892974  0.267577
CHits      0.138303 -0.150033
CHmRun    -0.033312  0.035797
CRuns      1.132763  0.400545
CRBI       0.581068 -0.090109
CWalks    -0.475808  0.257955
League    -0.168323 -0.180426
Division   0.206751  0.234109
PutOuts    0.174039  0.171798
Assists    0.122905  0.029025
Errors    -0.052294 -0.092255
NewLeague  0.020377  0.123357


Seems like it does not match the glm very well

In [100]:
dir(keras.optimizers)

['Adadelta',
 'Adafactor',
 'Adagrad',
 'Adam',
 'AdamW',
 'Adamax',
 'Ftrl',
 'Lamb',
 'Lion',
 'LossScaleOptimizer',
 'Nadam',
 'Optimizer',
 'RMSprop',
 'SGD',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'deserialize',
 'get',
 'legacy',
 'schedules',
 'serialize']

In [101]:
modelnn.compile(loss='mean_squared_error', optimizer = 'Adagrad')
modelnn.fit(X, Y, epochs=200, verbose=0)
showparams['nn_adagrad'] = modelnn.get_weights()[0]

modelnn.compile(loss='mean_squared_error', optimizer = 'Adamax')
modelnn.fit(X, Y, epochs=200, verbose=0)
showparams['nn_Adamax'] = modelnn.get_weights()[0]


In [102]:
print(showparams)

                glm        nn  nn_adagrad  nn_Adamax
AtBat     -0.658632  0.022274    0.014089  -0.019052
Hits       0.755723 -0.091555   -0.081175  -0.041370
HmRun      0.080603 -0.314586   -0.312970  -0.298799
Runs      -0.121588  0.023879    0.029169   0.052286
RBI       -0.047869  0.586889    0.573196   0.516353
Walks      0.291328  0.002488    0.002313   0.009681
Years     -0.021399 -0.290138   -0.284870  -0.266854
CAtBat    -0.892974  0.267577    0.266789   0.258454
CHits      0.138303 -0.150033   -0.145543  -0.132094
CHmRun    -0.033312  0.035797    0.041656   0.065285
CRuns      1.132763  0.400545    0.401890   0.404398
CRBI       0.581068 -0.090109   -0.083940  -0.059121
CWalks    -0.475808  0.257955    0.242864   0.182348
League    -0.168323 -0.180426   -0.180957  -0.179341
Division   0.206751  0.234109    0.235413   0.243338
PutOuts    0.174039  0.171798    0.170969   0.171625
Assists    0.122905  0.029025    0.027902   0.025951
Errors    -0.052294 -0.092255   -0.089661  -0.

They only vary a little betwwen different optimizers, but still far off the glm, which is weird.

## Problem 2

#### part a)

In [103]:
asd = 1.a

SyntaxError: invalid decimal literal (2995272198.py, line 1)

In [ ]:
import statsmodels.api as sm

In [ ]:
default = pd.read_csv('C:\\Users\\lorentsen\\Documents\\Python\\Math-180-Applied-Statistics-and-Machine-Learning\\ISLRv2_data\\ALL CSV FILES - 2nd Edition\\Default.csv')

default.head()
default['default'] = default['default'].map({'No': 0, 'Yes': 1})
default['student'] = default['student'].map({'No': 0, 'Yes': 1})

X = MS(default.columns.drop('default'), intercept = False).fit_transform(default)
Y = default['default']

GLMmodel = sm.GLM(
    Y
    , X
    , family = sm.families.Binomial()
    )
regr = GLMmodel.fit()
regr.params

student   -3.753126
balance    0.002795
income    -0.000174
dtype: float64

#### part b) 
See pdf
#### part c)


In [ ]:
modelnn = keras.Sequential(
    [
        layers.Dense(2, activation = 'softmax', input_shape = (X.shape[1],))
    ]
)

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### part e)

In [ ]:
modelnn.compile(loss='categorical_crossentropy', optimizer = 'adam')
modelnn.fit(X, keras.utils.to_categorical(Y), epochs=200, verbose=0)

<!-- #### part f) -->

In [ ]:
compareparams = pd.DataFrame()
compareparams['glm'] = regr.params
compareparams['nn'] = np.diff(modelnn.get_weights()[0], axis = 1)
compareparams

,glm,nn
student,-3.753126,-11.810568
balance,0.002795,0.018515
income,-0.000174,-0.000816


They are fairly similar

## Problem 3
### part a)

In [ ]:
4.5+np.log(0.99)

4.489949664146499

In [ ]:
3/17*10

1.7647058823529413

In [ ]:
5*0.15 + 1*0.25 

1.0